In [11]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [12]:
%%R
testData = data.matrix(read.csv( file="test.csv", header=TRUE ))

In [13]:
%%R
print(head(testData))

     bidder_id payment_account address
[1,]      1345            2638    1338
[2,]      3105            3345    2890
[3,]      1978            1260    2650
[4,]      4322            3038    3530
[5,]      3800            2376    3774
[6,]      3895            3568    2145


In [14]:
import csv
from sets import Set

#build a set of bidder ids

print('starting ...')

fin = open('test.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

bidderIdSet = Set([])

for row in rdr:
    bidderId = row[0]
    if bidderId == 'bidder_id': #skip header
        continue
    bidderIdSet.add(bidderId)

print(len(bidderIdSet))

fin.close()

starting ...
4700


In [15]:
import csv

#separate bids for test bidders

print('starting ...')

fin = open('bids.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

testFout = open('test_bids.csv', 'wb')
testWtr = csv.writer(testFout, delimiter=',')

#write only the bidders which exist in test dataset
for row in rdr:
    bidderId = row[1]
    if row[0] == 'bid_id': #header
        testWtr.writerow(row)
    elif bidderId in bidderIdSet:
        testWtr.writerow(row)
        
fin.close()
testFout.close()

print('done')

starting ...
done


In [17]:
import csv

#process test bidders info

fin = open('test_bids.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

# things to consider for each bidder
# -arrival distribution
# -max rate
# -total number of bids
# -number of devices used
# -number of auctions used
# -number of different merchandises (categories)
# -number of countries accessed 


bidderIdToInfo = {} #bidderId -> info object

for row in rdr:
    bidderId = row[1]
    if bidderId == 'bidder_id':
        continue
    elif bidderIdToInfo.has_key(bidderId):
        bidderIdToInfo[bidderId]['numBids'] = bidderIdToInfo[bidderId]['numBids'] + 1
        arrivalTimesList = bidderIdToInfo[bidderId]['arrivalTimes']
        arrivalTimesList.append(row[5]) #append the new arrival time
        length = len(arrivalTimesList)
        bidderIdToInfo[bidderId]['avgRate'] = (int(row[5])-int(bidderIdToInfo[bidderId]['arrivalTimes'][0])) / (length-1)
    else:
        newInfo = {}
        newInfo['numBids'] = 1
        arrivalTimesList = []
        arrivalTimesList.append(row[5]) #append the arrival time of the current bid
        newInfo['arrivalTimes'] = arrivalTimesList
        newInfo['avgRate'] = '' #put -1 for test dataset ???
        bidderIdToInfo[bidderId] = newInfo
        
fin.close()
        
fout = open('test_bidders_info.csv', 'wb')
wtr = csv.writer(fout, delimiter=',')

wtr.writerow(['bidder_id', 'number of bids', 'avg arrival interval', 'arrival times'])
for key, val in bidderIdToInfo.items():
    currRowVec = [key, str(val['numBids']), val['avgRate'], ','.join(val['arrivalTimes'])]
    wtr.writerow(currRowVec)
    
fout.close()

print('Test bidders: ' + str(len(bidderIdToInfo)))

Test bidders: 4630
